In [2]:
%load_ext autoreload
%autoreload 2

from util_0801 import *
from myalgorithm_0801_1 import algorithm

## 실험 내용
- 크루스칼 돌릴 때 edge를 1/10 정도만 돌렸을 때 나오는 가중치 조합을 이용 가능한가?

### 기존 크루스칼 코드 결과 확인

In [26]:
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_2.json'
problem_file = '../alg_test_problems_20240429/TEST_K50_1.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

for v in ALL_RIDERS:
    print(v)

print(K)

Rider([BIKE, 5.291005291005291, 100, 60, 5000, 120, 10])
Rider([WALK, 1.3227513227513228, 70, 30, 5000, 120, 15])
Rider([CAR, 4.2328042328042335, 200, 100, 5000, 180, 50])
50


In [68]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    to_checks = []

    for weight1 in [0, 1, 2]:
        for weight2 in [-4, -3, -2, -1, 0]:
            avg_method = 'two'
            bundle_merging_function = try_merging_bundles_by_dist
            default_get_dist_function = get_dist_by_coords

            inf = float('inf')

            car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
            bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
            walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

            all_bundles = []
            for ord in ALL_ORDERS:
                new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
                car_rider.available_number -= 1
                all_bundles.append(new_bundle)

            # 2개 주문 묶음 생성
            all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

            # 4개 주문 묶음 생성
            all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

            # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
            new_all_bundles = []
            for bundle in all_bundles:
                if len(bundle.shop_seq) >= 3:
                    new_all_bundles.append(bundle)
                else:
                    old_rider = bundle.rider
                    old_rider.available_number += 1
                    for order_num in bundle.shop_seq:
                        order = ALL_ORDERS[order_num]

                        new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                        car_rider.available_number -= 1
                        new_all_bundles.append(new_bundle)

            all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

            ## ------------------- 라이더 재배치 -------------------------------

            all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
            for rider_i in range(3):
                ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

            ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

            solution = [
                    # rider type, shop_seq, dlv_seq
                    [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
                    for bundle in all_bundles
            ]

            with open(problem_file, 'r') as f:
                prob = json.load(f)

            K = prob['K']

            ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
            ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

            DIST = np.array(prob['DIST'])
            for r in ALL_RIDERS:
                r.T = np.round(DIST/r.speed + r.service_time).astype(int)

            checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    

In [50]:
for to_check in to_checks:
    print(len(to_check[0]), to_check[1], to_check[2], to_check[3], to_check[4])

14 31 0 -4 3450.912999999999
13 33 0 -3 3467.9769999999994
14 32 0 -2 3469.1999999999994
15 32 0 -1 3450.12
12 34 0 0 3596.32
16 30 1 -4 3515.474
13 32 1 -3 3520.5500000000006
16 30 1 -2 3464.946000000001
14 32 1 -1 3480.178
16 30 1 0 3428.4920000000006
14 33 2 -4 3686.3900000000017
16 32 2 -3 3629.3240000000014
14 31 2 -2 3635.3220000000006
15 33 2 -1 3586.9400000000005
15 31 2 0 3689.2580000000003


### 크루스칼 코드 변형 결과 확인

In [27]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## ------------------  크루스칼 함수   -----------------------------

edge_num_to_divide = 10

def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            if avg_method == 'avg':
                dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
            elif avg_method == 'two_seq':
                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]
            elif avg_method == 'two':
                order_num1 = all_bundles[i].shop_seq[0]
                order_num2 = all_bundles[j].shop_seq[0]

                dist1 = DIST[order_num1][order_num2]
                dist2 = DIST[order_num1 + K][order_num2 + K]  
            else:
                assert False

            # weight1 = (dist1 + dist2) / 900

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(len(all_bundles)))
    edges.sort(key=lambda x: x[2])
    edges = edges[:int(len(edges) // edge_num_to_divide)]

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue

        new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    rider_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, rider_availables

## --------------- 초기 번들링 최적화 코드 --------------------------

to_checks = []

for weight1 in [0, 0.5, 1, 1.5, 2]:
    for weight2 in [-5, -4.5, -4, -3.5, -3, -2.5, -2, -1.5, -1, 0.5, 0]:
        avg_method = 'two'
        bundle_merging_function = try_merging_bundles_by_dist
        default_get_dist_function = get_dist_by_coords

        inf = float('inf')

        car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
        bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
        walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

        all_bundles = []
        for ord in ALL_ORDERS:
            new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
            car_rider.available_number -= 1
            all_bundles.append(new_bundle)

        # 2개 주문 묶음 생성
        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

        # 4개 주문 묶음 생성
        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

        to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

        # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
        new_all_bundles = []
        for bundle in all_bundles:
            if len(bundle.shop_seq) >= 3:
                new_all_bundles.append(bundle)
            else:
                old_rider = bundle.rider
                old_rider.available_number += 1
                for order_num in bundle.shop_seq:
                    order = ALL_ORDERS[order_num]

                    new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                    car_rider.available_number -= 1
                    new_all_bundles.append(new_bundle)

        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

        ## ------------------- 라이더 재배치 -------------------------------

        all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
        for rider_i in range(3):
            ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

        ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

        solution = [
                # rider type, shop_seq, dlv_seq
                [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
                for bundle in all_bundles
        ]

        with open(problem_file, 'r') as f:
            prob = json.load(f)

        K = prob['K']

        ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
        ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

        DIST = np.array(prob['DIST'])
        for r in ALL_RIDERS:
            r.T = np.round(DIST/r.speed + r.service_time).astype(int)

        checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

        to_checks.append((to_check, len(all_bundles), weight1, weight2, checked_solution['avg_cost']))

In [24]:
for to_check in to_checks:
    print(len(to_check[0]), to_check[1], to_check[2], to_check[3], to_check[4])

7 47 0 -5 4136.462
7 43 0 -4.5 3948.3450000000007
7 42 0 -4 3901.3550000000005
5 41 0 -3.5 3797.9339999999997
6 41 0 -3 3794.243
9 38 0 -2.5 3614.6459999999997
10 37 0 -2 3613.93
8 37 0 -1.5 3623.002000000001
7 41 0 -1 3817.824
12 39 0 0.5 3850.263
11 36 0 0 3696.502
8 41 0.5 -5 3835.988
8 41 0.5 -4.5 3919.5229999999992
8 39 0.5 -4 3805.4489999999996
9 38 0.5 -3.5 3764.2690000000002
13 35 0.5 -3 3614.896
11 35 0.5 -2.5 3601.400000000001
12 33 0.5 -2 3480.6980000000012
13 34 0.5 -1.5 3513.1440000000007
11 35 0.5 -1 3546.1380000000004
14 33 0.5 0.5 3497.9760000000006
13 34 0.5 0 3510.5240000000003
11 37 1 -5 3667.987
11 37 1 -4.5 3693.4760000000006
12 36 1 -4 3664.69
13 37 1 -3.5 3707.5800000000004
12 36 1 -3 3664.8140000000008
13 36 1 -2.5 3719.764000000001
14 34 1 -2 3615.4760000000015
13 32 1 -1.5 3500.720000000001
13 34 1 -1 3518.2260000000006
12 35 1 0.5 3750.8800000000006
13 32 1 0 3494.9320000000002
11 36 1.5 -5 3747.465
11 38 1.5 -4.5 3822.456000000001
10 37 1.5 -4 3759.904000000

### 데이터셋별 크루스칼 코드 속도 및 비용 확인

In [8]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    start_time = time.time()

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    weight1 = 1
    weight2 = -2

    avg_method = 'two'
    bundle_merging_function = try_merging_bundles_by_dist
    default_get_dist_function = get_dist_by_coords

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    end_time = time.time()

    process_time = end_time - start_time

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    results.append((process_time, K, checked_solution['avg_cost'], testcase_name))

print(results)

[(0.39191532135009766, 50, 3698.22, 'TEST_K50_1'), (0.5013291835784912, 50, 4150.933999999999, 'TEST_K50_2'), (1.0874197483062744, 100, 3513.885, 'TEST_K100_1'), (2.25848126411438, 100, 3879.935, 'TEST_K100_2'), (5.2576658725738525, 200, 3434.051500000001, 'TEST_K200_1'), (5.533364295959473, 200, 3371.1545000000006, 'TEST_K200_2'), (1.3604977130889893, 100, 3903.302, 'STAGE1_1'), (0.30835723876953125, 100, 3464.946000000001, 'STAGE1_2'), (4.389868497848511, 200, 3185.4685, 'STAGE1_3'), (0.7303218841552734, 200, 2781.807999999999, 'STAGE1_4'), (3.032329559326172, 300, 2328.6326666666673, 'STAGE1_5'), (1.2622816562652588, 300, 2713.6219999999994, 'STAGE1_6'), (1.1081798076629639, 100, 3671.8979999999997, 'STAGE1_7'), (0.32689332962036133, 100, 3451.152, 'STAGE1_8'), (5.938086986541748, 200, 3173.309, 'STAGE1_9'), (0.5914301872253418, 200, 2688.8149999999987, 'STAGE1_10'), (6.789900064468384, 300, 2573.1883333333335, 'STAGE1_11'), (1.411278247833252, 300, 2607.4796666666666, 'STAGE1_12'),

In [10]:
for tm, K, cost, name in results:
    print(tm, K, cost, name)

0.39191532135009766 50 3698.22 TEST_K50_1
0.5013291835784912 50 4150.933999999999 TEST_K50_2
1.0874197483062744 100 3513.885 TEST_K100_1
2.25848126411438 100 3879.935 TEST_K100_2
5.2576658725738525 200 3434.051500000001 TEST_K200_1
5.533364295959473 200 3371.1545000000006 TEST_K200_2
1.3604977130889893 100 3903.302 STAGE1_1
0.30835723876953125 100 3464.946000000001 STAGE1_2
4.389868497848511 200 3185.4685 STAGE1_3
0.7303218841552734 200 2781.807999999999 STAGE1_4
3.032329559326172 300 2328.6326666666673 STAGE1_5
1.2622816562652588 300 2713.6219999999994 STAGE1_6
1.1081798076629639 100 3671.8979999999997 STAGE1_7
0.32689332962036133 100 3451.152 STAGE1_8
5.938086986541748 200 3173.309 STAGE1_9
0.5914301872253418 200 2688.8149999999987 STAGE1_10
6.789900064468384 300 2573.1883333333335 STAGE1_11
1.411278247833252 300 2607.4796666666666 STAGE1_12
1.9995548725128174 100 5033.758 STAGE1_13
0.6888482570648193 100 4426.475 STAGE1_14
7.794760227203369 200 4410.554999999999 STAGE1_15
0.91439557

#### 가중치 확인 시간 30초 안넘게 해보기

In [11]:
30 / 0.5623748302459717

53.34520392187287

In [ ]:
weight1_grid = [0, 0.5, 1, 1.5]
weight2_grid = [-3, -2.5, -2, -1.5, -1, -0.5, 0]

In [ ]:
weight1, weight2 in [(1, -1), (1, -2), (1, -3), (1, -4), (1, -5), (1, -0.5), (1, -1.5),]

In [12]:
weights = []

for weight1 in [1, 0.5, 0, 1.5]:
    for weight2 in [-1, -2, 0, -3, -4, -5, -0.5, -1.5, -2.5]:
        weights.append((weight1, weight2))

for weight1 in [1, 0.5, 0, 1.5]:
    for weight2 in [-3.5, -4.5, -5.5]:
        weights.append((weight1, weight2))    

In [14]:
print(weights)

[(1, -1), (1, -2), (1, 0), (1, -3), (1, -4), (1, -5), (1, -0.5), (1, -1.5), (1, -2.5), (0.5, -1), (0.5, -2), (0.5, 0), (0.5, -3), (0.5, -4), (0.5, -5), (0.5, -0.5), (0.5, -1.5), (0.5, -2.5), (0, -1), (0, -2), (0, 0), (0, -3), (0, -4), (0, -5), (0, -0.5), (0, -1.5), (0, -2.5), (1.5, -1), (1.5, -2), (1.5, 0), (1.5, -3), (1.5, -4), (1.5, -5), (1.5, -0.5), (1.5, -1.5), (1.5, -2.5), (1, -3.5), (1, -4.5), (1, -5.5), (0.5, -3.5), (0.5, -4.5), (0.5, -5.5), (0, -3.5), (0, -4.5), (0, -5.5), (1.5, -3.5), (1.5, -4.5), (1.5, -5.5)]


In [ ]:
[(1, -3), (1, -2.5), (1, -2), (1.5)]